<a href="http://datamics.com/de/courses/"><img src=../DATA/bg_datamics_top.png></a>

<em text-align:center>Copyright Datamics</em>
# Mit eigenen Bildern arbeiten

Bisher waren alle unsere verwendeten Daten bereits von Keras für uns formatiert.

Lass uns erkunden, wie es ist, mit einem realistischerem Datensatz zu arbeiten.


## Die Daten

-----------

## BITTE BEACHTE: DIESER DATENSATZ IST SEHR GROSS. ER KANN AUS DER VORHERIGEN LEKTION HERUNTERGELADEN WERDEN. BITTE SIEH DIR DAS VIDEO AN FÜR INFORMATIONEN ZUM ERHALT DER DATEN.

## VERWENDE UNSERE VERSION DER DATEN. WIR HABEN BEREITS ALLES FÜR DICH VORBEREITET!

--------
----------
--------

ORIGINALE DATENQUELLE:

https://www.microsoft.com/en-us/download/confirmation.aspx?id=54765

-----------

Der Kaggle-Wettbewerb: [Cats and Dogs](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition) enthält 25.000 Bilder von Katzen und Hunden. Wir werden einen Klassifizierer bauen, der mit diesen Bildern arbeitet und versucht, Katzen und Hunde zu unterscheiden!

Die Bilder sind von 0-12499 nummeriert, sowohl Katzen als auch Hunde, wir haben also 12.500 Hundebilder und 12.500 Katzenbilder. Das ist ein großer Datensatz!!


--------
----------
------------


**BEACHTE: Wir werden mit echten Bildern arbeiten, nicht Numpy-Arrays. Dies bedeutet, dass das Lernen, wie man mit einer großen Menge von Bilddateien arbeitet, ein großer Anteil dieses Prozesses sein wird. Es sind zu viele Daten, um sie im Arbeitsspeicher als Numpy-Array zu halten, daher werden wir sie in unser Modell in Schüben einladen müssen.**

### Daten Visualisieren


-------
Lass uns einen genaueren Blick auf die Daten werfen:

In [3]:
%pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [4]:
import matplotlib.pyplot as plt
import cv2
# Technisch gesehen nicht mehr nötig in der neusten Version von Jupyter
%matplotlib inline

In [5]:
cat4 = cv2.imread('../DATA/CATS_DOGS/train/CAT/4.jpg')
cat4 = cv2.cvtColor(cat4,cv2.COLOR_BGR2RGB)

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-9d_dfo3_\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
type(cat4)

In [ ]:
cat4.shape

In [ ]:
plt.imshow(cat4)

In [ ]:
dog2 = cv2.imread('../DATA/CATS_DOGS/train/Dog/2.jpg')
dog2 = cv2.cvtColor(dog2,cv2.COLOR_BGR2RGB)

In [ ]:
dog2.shape

In [ ]:
plt.imshow(dog2)

## Die Daten für das Modell vorbereiten

Es sind zu viele Daten für uns, um sie alle auf einmal in den Arbeitsspeicher zu laden. Wir können einige von Keras zur Verfügung gestellte Funktionen zum automatischen Verarbeiten der Dateien verwenden, so das Einlesen in Schüben aus einem Verzeichnis durchführen und auch die Bilder bearbeiten.

### Bildbearbeitung

Es ist im Allgemeinen eine gute Idee, die Bilder mit Rotation, Grössenanpassung und Skalierung zu bearbeiten, damit das Modell robuster wird für andere, nicht in unserem Datensatz enthaltene Bilder. Wir können hierzu den automatisierten **ImageDataGenerator** verwenden. Wirf einen Blick in die Dokumentation für eine vollständige Liste aller Parameter, die du hier verwenden kannst!

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
image_gen = ImageDataGenerator(rotation_range=30, # Rotiere das Bild um 90 Grad
                               width_shift_range=0.1, # Verschiebe die Bildbreite um maximal 10%
                               height_shift_range=0.1, # Verschiebe die Bildhöhe um maximal 10%
                               rescale=1/255, # Skaliere das Bild durch Normalisierung
                               shear_range=0.2, # Schneide ein Teil des Bildes ab (maximal 20%)
                               zoom_range=0.2, # Vergrössere das Bild um maximal 20%
                               horizontal_flip=True, # Erlaube das Spiegeln an der horizontalen Achse
                               fill_mode='nearest' # Fülle fehlende Pixel mit dem nächsten, ausgefüllten Wert
                              )

In [ ]:
plt.imshow(image_gen.random_transform(dog2))

In [ ]:
plt.imshow(image_gen.random_transform(dog2))

In [ ]:
plt.imshow(image_gen.random_transform(dog2))

### Viele manipulierte Bilder aus einem Verzeichnes generieren


Um .flow_from_directory zu verwenden, musst du die Bilder in Unterverzeichnissen vorbereiten. Dies ist eine absolute Vorraussetzung, andernfalls kann die Methode nicht funktionieren. Die Verzeichnisse sollten nur Bilder einer Klasse enthalten, also ein Verzeichnis pro Bildklasse.

Benötigte Struktur:

* Bilddatenordner
    * Klasse 1
        * 0.jpg
        * 1.jpg
        * ...
    * Klasse 2
        * 0.jpg
        * 1.jpg
        * ...
    * ...
    * Klasse n

In [ ]:
image_gen.flow_from_directory('../DATA/CATS_DOGS/train')

In [ ]:
image_gen.flow_from_directory('../DATA/CATS_DOGS/test')

### Bildgrößenanpassung

Lasst uns mit Keras die Bildgröße aller Bilder auf 150 mal 150 Pixel setzen, nachdem sie bearbeitet wurden.

In [ ]:
# Breite, Höhe, Kanäle
image_shape = (150,150,3)

# Das Modell erzeugen

In [ ]:
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D

In [ ]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=(150,150,3), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=(150,150,3), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=(150,150,3), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())


model.add(Dense(128))
model.add(Activation('relu'))

# Dropouts (Ausfälle) verhindern ein overfitting (Überanpassung) durch das zufällige Ausschalten von Neuronen während des Trainings.
# Hier sagen wir: schalte zufällig 50% aller Neuronen aus.
model.add(Dropout(0.5))

# Letztes Layer (Ebene), wie zuvor ist es binär, 0=Katze , 1=Hund
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.summary()

### Modell trainieren

In [ ]:
batch_size = 16

train_image_gen = image_gen.flow_from_directory('../DATA/CATS_DOGS/train',
                                               target_size=image_shape[:2],
                                               batch_size=batch_size,
                                               class_mode='binary')

In [ ]:
test_image_gen = image_gen.flow_from_directory('../DATA/CATS_DOGS/test',
                                               target_size=image_shape[:2],
                                               batch_size=batch_size,
                                               class_mode='binary')

In [ ]:
train_image_gen.class_indices

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
results = model.fit_generator(train_image_gen,epochs=100,
                              steps_per_epoch=150,
                              validation_data=test_image_gen,
                             validation_steps=12)

In [ ]:
# model.save('cat_dog2.h5')

# Modell evaluieren

In [ ]:
results.history['acc']

In [ ]:
plt.plot(results.history['acc'])

In [ ]:
# model.save('cat_dog_100epochs.h5')

# Neue Bilder beurteilen

In [ ]:
train_image_gen.class_indices

In [ ]:
import numpy as np
from keras.preprocessing import image

dog_file = '../DATA/CATS_DOGS/train/Dog/2.jpg'

dog_img = image.load_img(dog_file, target_size=(150, 150))

dog_img = image.img_to_array(dog_img)

dog_img = np.expand_dims(dog_img, axis=0)
dog_img = dog_img/255

In [ ]:
prediction_prob = model.predict(dog_img)

In [ ]:
# Beurteilung ausgeben
print(f'Probability that image is a dog is: {prediction_prob} ')

# Gut gemacht!